In [35]:
import os
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import shutil
import random

from scipy import signal
from collections import OrderedDict

In [36]:
path = r'c:\Users\dhers\OneDrive - UBA\Club de la neurona\example_cdln.h5'

df = pd.read_hdf(path)
scorer = df.columns.levels[0][0]
bodyparts = df.columns.levels[1].to_list()

df_raw = df.copy()
df_raw

scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             217.767487  133.605804   
1                                             217.742615  133.606812   
2                                             217.745010  133.668839   
3                                             217.720123  133.702316   
4                                             217.725815  133.692230   
..                                                   ...         ...   
595                                           217.275970  133.738556   
596                                           217.325470  133.757004   
597                                           217.387421  133.744659   
598                                           217.391403  133.726303   
599                                           217.397079  133.752762   

scorer                                                               \
bodyparts                  obj_2                               nose   
coords    likelihood           x           y likelihood           x   
0           0.957475  484.346924  136.355408   0.948422  365.933258   
1           0.957617  484.353546  136.349533   0.950448  367.081818   
2           0.956313  484.357544  136.341324   0.949334  366.866425   
3           0.955090  484.334015  136.361313   0.945641  359.043549   
4           0.956116  484.392822  136.384476   0.945345  354.271759   
..               ...         ...         ...        ...         ...   
595         0.955184  484.199585  136.544174   0.958359  239.311295   
596         0.956045  484.167969  136.539719   0.957126  225.862030   
597         0.957283  484.189941  136.510651   0.954683  214.000031   
598         0.957166  484.273224  136.505112   0.954430  200.510956   
599         0.956024  484.299500  136.505188   0.954264  187.996765   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0          278.300842   0.873965  339.942322  ...   0.980906  227.464874   
1          275.349396   0.837746  340.856384  ...   0.950444  228.341583   
2          274.653381   0.835938  340.850769  ...   0.930455  228.493591   
3          281.609131   0.832095  336.592621  ...   0.957117  228.863480   
4          283.944122   0.874340  334.082733  ...   1.000000  227.563126   
..                ...        ...         ...  ...        ...         ...   
595        365.859467   0.877451  271.627197  ...   0.961922  404.476501   
596        368.296814   0.875561  255.574356  ...   0.902448  386.686188   
597        366.961792   0.885680  240.405197  ...   0.927450  372.306824   
598        363.545349   0.909760  225.635101  ...   0.953016  357.629669   
599        359.116608   0.985355  209.741531  ...   0.948782  341.303650   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0          340.451630   0.979043  156.781357  351.009979   0.867568   
1          340.312164   0.976494  158.498886  351.037262   0.840485   
2          339.951355   0.989730  158.586456  351.776703   0.829707   
3          339.238831   0.984237  159.294250  351.988312   0.837167   
4          340.027618   0.986425  157.454025  350.071228   0.876206   
..                ...        ...         ...         ...        ...   
595        380.882172   1.000000  475.078003  366.441620   0.960453   
596        373.478210   1.000000  458.410400  359.419434   0.966336   
597        373.532745   1.000000  443.978729  350.301208   0.948537   
598        377.041992   0.969050  429.198212  354.882904   0.967920   
599     

In [37]:
# Try different filtering parameters
med_window = 3
sigma, n_sigmas = 0.6, 2
N = int(2 * n_sigmas * sigma + 1)

# Gaussian kernel
gauss_kernel = signal.windows.gaussian(N, sigma)
gauss_kernel = gauss_kernel / sum(gauss_kernel)

pad_width = (len(gauss_kernel) - 1) // 2
gauss_kernel

array([0.16637851, 0.66724298, 0.16637851])

In [38]:
# Iterate over each body part
for point in bodyparts:

    mean = df[scorer, point, 'likelihood'].mean()
    std_dev = df[scorer, point, 'likelihood'].std()
    limit = mean - 2*std_dev

    print(f'{point}\t{limit}')
    
    # Create mask where likelihood is below the threshold
    mask = df[scorer, point, 'likelihood'] < limit
    # Set 'x' and 'y' to NaN where likelihood is below the threshold
    df.loc[mask, (scorer, point, ['x', 'y'])] = np.nan

    # Iterate over x and y axes
    for axis in ['x', 'y']:
        # Access the specific axis column for this body part
        # column = df[scorer, point, axis]

        # Check if there are any non-NaN values left to interpolate
        if df[scorer, point, axis].notna().sum() > 1:
            # Interpolate using the pchip method, limit_area='inside' ensures interpolation only within valid data
            df[scorer, point, axis] = df[scorer, point, axis].interpolate(method='pchip', limit_area='inside')
            
            # Forward fill the remaining NaN values
            df[scorer, point, axis] = df[scorer, point, axis].ffill()
            
            # Apply median filter
            df[scorer, point, axis] = signal.medfilt(df[scorer, point, axis], kernel_size=med_window)
            
            # Pad the median filtered data to mitigate edge effects
            padded = np.pad(df[scorer, point, axis], pad_width, mode='edge')
            # Apply convolution
            smooth = signal.convolve(padded, gauss_kernel, mode='valid')
            # Trim the padded edges to restore original length
            df[scorer, point, axis] = smooth[:len(df[scorer, point, axis])]
            
            if 'obj' in point:
                df[scorer, point, axis] = df[scorer, point, axis].median()
            
df

L_ear	0.8589539155364037
R_ear	0.8331150412559509
body	0.8584106490015984
head	0.7924715504050255
neck	0.8577349931001663
nose	0.5399865806102753
obj_1	0.9131294302642345
obj_2	0.9209924917668104
tail_1	0.790634274482727
tail_2	0.8347642049193382
tail_3	0.6503640413284302


scorer    DLC_Resnet50_STORM_2Oct14shuffle2_snapshot_900              \
bodyparts                                          obj_1               
coords                                                 x           y   
0                                             217.665195  133.581802   
1                                             217.665195  133.581802   
2                                             217.665195  133.581802   
3                                             217.665195  133.581802   
4                                             217.665195  133.581802   
..                                                   ...         ...   
595                                           217.665195  133.581802   
596                                           217.665195  133.581802   
597                                           217.665195  133.581802   
598                                           217.665195  133.581802   
599                                           217.665195  133.581802   

scorer                                                              \
bodyparts                  obj_2                              nose   
coords    likelihood           x          y likelihood           x   
0           0.957475  484.434609  136.38621   0.948422  366.088517   
1           0.957617  484.434609  136.38621   0.950448  366.711166   
2           0.956313  484.434609  136.38621   0.949334  365.564866   
3           0.955090  484.434609  136.38621   0.945641  359.551184   
4           0.956116  484.434609  136.38621   0.945345  354.855993   
..               ...         ...        ...        ...         ...   
595         0.955184  484.434609  136.38621   0.958359  240.297481   
596         0.956045  484.434609  136.38621   0.957126  226.126117   
597         0.957283  484.434609  136.38621   0.954683  213.729320   
598         0.957166  484.434609  136.38621   0.954430  200.673156   
599         0.956024  484.434609  136.38621   0.954264  190.078858   

scorer                                        ...                         \
bodyparts                              L_ear  ...       body      tail_1   
coords              y likelihood           x  ... likelihood           x   
0          275.349396   0.873965  340.093468  ...   0.980906  227.610740   
1          275.349396   0.837746  340.699623  ...   0.950444  228.221009   
2          276.390881   0.835938  340.142305  ...   0.930455  228.468300   
3          280.750302   0.832095  336.883494  ...   0.957117  228.338782   
4          282.730149   0.874340  334.281704  ...   1.000000  227.505580   
..                ...        ...         ...  ...        ...         ...   
595        365.576829   0.877451  271.868232  ...   0.961922  404.232301   
596        366.778389   0.875561  255.721382  ...   0.902448  387.253696   
597        366.393369   0.885680  240.471593  ...   0.927450  372.257278   
598        363.376924   0.909760  225.448179  ...   0.953016  357.355334   
599        359.853455   0.985355  212.385880  ...   0.948782  344.019949   

scorer                                                               \
bodyparts                             tail_2                          
coords              y likelihood           x           y likelihood   
0          340.312164   0.979043  157.067117  351.014519   0.867568   
1          340.252133   0.976494  158.308156  351.113795   0.840485   
2          340.011386   0.989730  158.975023  351.443464   0.829707   
3          339.964044   0.984237  158.801181  351.282737   0.837167   
4          340.180252   0.986425  157.500718  350.313027   0.876206   
..                ...        ...         ...         ...        ...   
595        380.900151   1.000000  475.109844  366.000413   0.960453   
596        374.755532   1.000000  458.782411  359.832993   0.966336   
597        374.116609   1.000000  443.920689  355.637685   0.948537   
598        376.458129   0.969050  429.271002  354.882904   0.967920   
599        377.041992 

In [39]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(x=df.index, y=df[scorer]['nose']['x']))
fig.add_trace(go.Scatter(x=df_raw.index, y=df_raw[scorer]['nose']['x']))

In [34]:
# Save the updated DataFrame to an HDF5 file
out_path = r'c:\Users\dhers\OneDrive - UBA\Club de la neurona\example_cdln.h5'
# df_raw.to_hdf(out_path, key='df', mode='w')